[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/diogoflim/MGP/blob/main/GP/Multicritério/mcda_2.ipynb)


In [ ]:
import pandas as pd
import numpy as np

# Modelagem e Gestão de Processos

TEP - UFF


**Professor: Diogo Ferreira de Lima Silva**

## Análise de Fornecedores

Uma organização deseja selecionar os melhores fornecedores para a realização de uma certa atividade. Para isso, seis critérios foram considerados e as avaliações dos fornecedores são dadas na tabela abaixo.

| Fornecedores | Preço         | Qualidade | Confiabilidade | Credibilidade | Assistência | Lead time |
|--------------|---------------|-----------|-----------------|---------------|-------------|-----------|
| Fornecedor 1 | 1.834.600,20  | 1         | 18              | 90            | 1           | 30        |
| Fornecedor 2 | 1.823.240,00  | 2         | 5               | 100           | 0           | 30        |
| Fornecedor 3 | 1.902.311,00  | 3         | 10              | 80            | 1           | 30        |
| Fornecedor 4 | 1.795.200,00  | 1         | 9               | 95            | 0           | 30        |
| Fornecedor 5 | 1.712.000,50  | 2         | 1               | 100           | 0           | 35        |
| Fornecedor 6 | 1.697.890,25  | 3         | 16              | 90            | 1           | 45        |
| Fornecedor 7 | 1.722.765,33  | 2         | 4               | 85            | 1           | 40        |

Fonte: [artigo_sbpo](https://proceedings.science/sbpo/sbpo-2022/trabalhos/uso-do-metodo-fitradeoff-para-selecao-de-fornecedores-em-um-processo-licitatorio?lang=pt-br)

Aplique o método ELECTRE I para encontrar o conjunto de alternativas não dominadas (kernel).

In [ ]:
data = {
    'Fornecedores': ['Fornecedor 1', 'Fornecedor 2', 'Fornecedor 3', 'Fornecedor 4', 'Fornecedor 5', 'Fornecedor 6', 'Fornecedor 7'],
    'Preço': [-1834600.20, -1823240.00, -1902311.00, -1795200.00, -1712000.50, -1697890.25, -1722765.33],
    'Qualidade': [1, 2, 3, 1, 2, 3, 2],
    'Confiabilidade': [18, 5, 10, 9, 1, 16, 4],
    'Credibilidade': [90, 100, 80, 95, 100, 90, 85],
    'Assistência': [1, 0, 1, 0, 0, 1, 1],
    'Lead time': [-30, -30, -30, -30, -35, -45, -40]
}

df = pd.DataFrame(data).set_index("Fornecedores")
df

# Decisão Multicritério

A literatura de decisão multicritério apresenta uma vasta quantidade de métodos para a resolução de problemas de decisão que envolvem múltiplos objetivos, muitas vezes conflitantes.

A Biblioteca PyDecision (https://github.com/Valdecy/pyDecision) inclui diversos métodos em Python. 

In [ ]:
# No Google Colab
!pip install pyDecision

# ELECTRE I

Para resolver um problema de decisão que se enquadra na problemática da escolha, pode-se aplicar o método ELECTRE I.

Os métodos da família ELECTRE apresentam características interessantes. São não compensatórios, apresentam comparações par-a-par e exploração de relações de sobreclassificação.

No caso do ELECTRE I, busca-se alcançar um kernel de alternativas não dominadas.

Os únicos parâmetros desse método incluem os pesos dos critérios $W = [w_1, w_2, ..., w_n]$, e os limiares de concordância ($\hat{c}$) e discordância ($\hat{d}$).

Inicialmente, vamos importar a biblioteca e o método correspondente:

In [ ]:
from pyDecision.algorithm import electre_i

Em seguida, passaremos a nossa matriz de decisão para o formato adequado de leitura da biblioteca.

A biblioteca PyDecision utiliza **numpy** arrays, assim, a conversão é bastante simples:

In [ ]:
X = np.array(df)

X

Vamos definir nosso vetor de pesos:

In [ ]:
W = [0.2, 0.1, 0.1, 0.2, 0.2, 0.2]

Agora, só nos resta definir os limiares de concordância e discordância:

In [ ]:
c_hat = 0.7
d_hat = 0.4

### Aplicação do ELECTRE I

Vamos aplicar a função específica abaixo. Perceba que o grafo correspondente às relações de sobreclassificação será impresso. 

In [ ]:
concordance, discordance, dominance, kernel, dominated = electre_i(X, W = W, remove_cycles = True, c_hat = c_hat, d_hat = d_hat, graph = True)

In [ ]:
dominance # Matriz de relações de sobreclassificação

In [ ]:
kernel # kernel

**Solução:** os fornecedores 5 e 6 seriam selecionados. Perceba que as colunas correspondentes não possuem valores 1.

## Exercício


Faça alterações nos limiares c_hat e d_hat. Verifique se as relações permanecem as mesmas. 

# ELECTRE TRI


Para problemas de **classificação**, a família de métodos ELECTRE apresenta o ELECTRE TRI. 

Em sua versão clássica, é também conhecido por ELECTRE TRI-B. 

A letra "B" significa boundary, indicando que o método usa perfis de fronteira entre as classes.



In [ ]:
from pyDecision.algorithm import electre_tri_b

Para a problemática de classificação, vamos usar o exemplo da análise de países em acordo com os indicadores do Legathum Prosperity Index.

In [ ]:
url = 'https://raw.githubusercontent.com/diogoflim/ProjIntegrador_PO_IA/main/Dados/LegathumProsperityIndex.csv'
df = pd.read_csv(url, index_col=0, sep = ';')

df.iloc[:10]

In [ ]:
X = np.array(df)
X

No método ELECTRE TRI, além dos pesos dos critérios, precisamos indicar se existem:

- Limiares de indiferença;
- Limiares de preferência;
- Limiares de veto;
- Nível de corte;

Além desses parâmetros, o decisor precisa indicar perfis que ilustram a fronteira entre cada duas classes consecutivas.

Digamos que nosso problema possui 3 classes, onde: $C_0 \succ C_1 \succ C_2$. 

Nesse caso, devem ser estabelecidos **dois perfis de fronteira**. 

- Um perfil indicará a fronteira entre as classes $C_0$ e $C_1$.
- Um perfil indicará a fronteira entre as classes $C_1$ e $C_2$.



In [ ]:
W = [1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12] #pesos

Q = [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5] # limiares de indiferença
P = [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10] # limiares de preferência
V = [50,  50,  50, 50,  50, 50, 50, 50, 50, 50, 50, 50] # limiares de veto

lambda_ = 0.75 # cut level

# Perfis de Fronteira
B = [[50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50], 
     [80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80]]

In [ ]:
classification = electre_tri_b(X, W , Q , P , V , B, cut_level = lambda_ , verbose = False, rule = 'oc', graph = True)

In [ ]:
classification

In [ ]:
classification_df = pd.DataFrame(classification, columns=["Classe"], index = df.index)
classification_df

In [ ]:
classification_df.value_counts()

In [ ]:
classification_df[classification_df["Classe"]==0]

In [ ]:
classification_df[classification_df["Classe"]==2]

## Exercício

Aplique a classificação usando a regra pessimista. Os resultados foram alterados? Comente.

## Exercício

Aumente o número de classes para 5.

Nesse caso, você precisará definir 4 perfis de fronteira!

Analise os resultados